In [12]:
# Modules to Import

import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow import keras 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from joblib import dump
from tensorflow.keras.callbacks import Callback, EarlyStopping, ReduceLROnPlateau

In [2]:
# Load Dataset

train_dataset=pd.read_csv("train.csv")
train_dataset

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41996,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41997,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41998,6,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
# Drop the Label Column to make training examples

X=train_dataset.drop(['label'],axis=1)

In [4]:
# Training Labels

y=train_dataset['label']

In [5]:
# Split dataset into train/test for evaluation

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=42)

In [7]:
# Converting pandas dataframe to numpy for keras 

x_train=x_train.to_numpy()
y_train=y_train.to_numpy()
x_test=x_test.to_numpy()
y_test=y_test.to_numpy()

In [8]:
# Making label one_hot_encoding

y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

# Normalizing the data 

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

In [9]:
x_train.shape

(37800, 784)

In [16]:
callbacks=[EarlyStopping(monitor = "val_loss", patience = 15, 
            restore_best_weights = True, mode='min')
            ,ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, mode='min')
            ]

In [15]:
# Model structure and hyperparameters

batch_size = 32
num_classes = 10
epochs = 30
model = Sequential()
model.add(Dense(512,activation='relu'))
# model.add(Dense(256,activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(128,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64,activation='relu'))
model.add(Dense(num_classes, activation='softmax'))
model.compile(loss=keras.losses.categorical_crossentropy,optimizer=keras.optimizers.Adam(learning_rate=0.001),metrics=['accuracy'])
hist = model.fit(x_train, y_train,batch_size=batch_size,epochs=epochs,verbose=1,validation_data=(x_test, y_test),callbacks=callbacks)
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Epoch 1/40
1182/1182 [==============================] - 16s 12ms/step - loss: 0.4217 - accuracy: 0.8703 - val_loss: 0.1549 - val_accuracy: 0.9500 - lr: 0.0010
Epoch 2/40
1182/1182 [==============================] - 15s 13ms/step - loss: 0.2033 - accuracy: 0.9402 - val_loss: 0.1258 - val_accuracy: 0.9624 - lr: 0.0010
Epoch 3/40
1182/1182 [==============================] - 15s 13ms/step - loss: 0.1589 - accuracy: 0.9525 - val_loss: 0.1138 - val_accuracy: 0.9664 - lr: 0.0010
Epoch 4/40
1182/1182 [==============================] - 15s 12ms/step - loss: 0.1392 - accuracy: 0.9589 - val_loss: 0.1010 - val_accuracy: 0.9707 - lr: 0.0010
Epoch 5/40
1182/1182 [==============================] - 14s 12ms/step - loss: 0.1229 - accuracy: 0.9636 - val_loss: 0.0984 - val_accuracy: 0.9695 - lr: 0.0010
Epoch 6/40
1182/1182 [==============================] - 14s 12ms/step - loss: 0.1085 - accuracy: 0.9685 - val_loss: 0.0990 - val_accuracy: 0.9707 - lr: 0.0010
Epoch 7/40
1182/1182 [========================

In [27]:
# Saving the model so that it can be used later

dump(model,'recognize.joblib')

Keras weights file (<HDF5 file "variables.h5" (mode r+)>) saving:
...layers\dense
......vars
.........0
.........1
...layers\dense_1
......vars
.........0
.........1
...layers\dense_2
......vars
.........0
.........1
...layers\dense_3
......vars
.........0
.........1
...layers\dropout
......vars
...layers\dropout_1
......vars
...metrics\mean
......vars
.........0
.........1
...metrics\mean_metric_wrapper
......vars
.........0
.........1
...optimizer
......vars
.........0
.........1
.........10
.........11
.........12
.........13
.........14
.........15
.........16
.........2
.........3
.........4
.........5
.........6
.........7
.........8
.........9
...vars
Keras model archive saving:
File Name                                             Modified             Size
config.json                                    2023-02-12 21:04:26         3059
metadata.json                                  2023-02-12 21:04:26           64
variables.h5                                   2023-02-12 21:04:2

['recognize.joblib']